# Working with Excel and CSV files in Kodexa

Kodexa allows you to work with a variety of document types, and this notebook will demonstrate how easy it is to access data from an Excel file.  We will be using concepts introduced in the "Getting Started" notebooks, so you may want to review/refer to them as you proceed through this example.

By the end of this example, you'll be able to parse an Excel workbook, access data on different sheets, and extract specific tables within the notebook.  We'll then show you how to export that specific data to a datastore that you could use for further processing.

All of our processing will occur in Kodexa's cloud environment.  In order to access the platform, you'll need to register for an account and generate an access token.  If you haven't done that already, follow the steps in our [Getting Started](https://developer.kodexa.com/kodexa-cloud/accessing-kodexa-cloud) guide.


Let's start with the imports:
1. Since all of our actions occur in the cloud, we'll need to import the KodexaPlatform and RemoteAction modules
2. Import Kodexa's Pipeline module so we can build pipelines and process our document
3. The Excel file we'll be processing is located in a file folder, so we'll import the FolderConnector in order to access it.
4. All files that have been processed/parsed in Kodexa (Excel, PDF, etc) become Kodexa Documents, so we'll import that module as well.

We're also setting the CLOUD_URL value to the platform environment on which we want to perform our processing


In [1]:
# The kodexa package is public
from kodexa import Document, Pipeline, FolderConnector, KodexaPlatform, RemoteAction

CLOUD_URL = 'https://platform.kodexa.com' 

## Set Platform Environment and Access Token Credential

In the next cell, you'll be prompted to enter your access token that you've created in the environment specified by the CLOUD_URL.
If you haven't created a token already, follow the steps in our [Getting Started](https://developer.kodexa.com/org-management/manage-access-token) guide.

* Note:  The text you enter in the prompt field will be masked.  Once you're done entering the access token value, hit enter to complete the action in the cell.  **You will then need to manulally set control at the next cell.**

In [2]:
import getpass

ACCESS_TOKEN = getpass.getpass("Enter access token:")

KodexaPlatform.set_url(CLOUD_URL)
KodexaPlatform.set_access_token(ACCESS_TOKEN)

Enter access token: ································


In [3]:
import os

# Setting up location of data file
DATA_FOLDER = '_data'
EXCEL_FOLDER = 'excel_workbooks'
DATA_FILE = '2019_Business_Expenses.xlsx'

FULL_PATH = os.path.join(os.getcwd(), '..', DATA_FOLDER, EXCEL_FOLDER, DATA_FILE)

print(f'\nThis is where the Excel document is located: {FULL_PATH}\n')



This is where the Excel document is located: /home/skep/Projects/Kodexa/kodexa-demo-notebooks/2_Examples_by_Source_Type/../_data/excel_workbooks/2019_Business_Expenses.xlsx



In [4]:

# Create a pipeline in order to access the Excel file
pipeline = Pipeline.from_file(FULL_PATH)

# Our first step in the Pipeline is to parse the Excel file.  
# The parser will produce a Kodexa document, which we'll retrive after the pipeline runs
pipeline.add_step(RemoteAction(slug='kodexa/excel-parser', options={}, attach_source=True))

# Do it!
pipeline.run()

# Let's get that freshly parsed document!
kodexa_doc = pipeline.context.output_document


## Let's inspect the document and see what we got back

Since this is a Kodexa document, all of the base methods and properties for the Document class are available to us.  We can access the metadata, get the content, etc.  Even though the source of this document was originally an Excel file, it still follows the same tree-like ContentNode structure as all Kodexa documents.

We can try it out here - let's view the content of the entire document


In [5]:
kodexa_doc.get_root().get_all_content()

"2019 Reimbursables UTILITIES Phone ISP Electricity Natural Gas Water Sewer Amt due from Business January 60.5 59.99 4.316046428571429 6.192921428571429 2.7588785714285717 133.75784642857144 February 60.5 59.99 3.5990035714285713 5.315514285714285 3.0407928571428573 132.4453107142857 March 57.79375 59.99 3.7471107142857147 5.297639285714286 2.674610714285714 129.5031107142857 April 60.512499999999996 59.99 3.318621428571429 0.6991678571428571 2.495860714285714 127.01615 May 60.512499999999996 69.99 3.3875678571428574 1.4126357142857144 2.651117857142857 137.95382142857144 June 60.5125 69.99 5.086714285714286 1.0694357142857145 3.8941964285714286 140.55284642857143 July 60.7125 69.99 9.156596428571428 0.9213285714285714 5.1970285714285716 145.97745357142855 August 60.53125 69.99 7.94365 0.9213285714285714 4.774667857142857 144.16089642857142 September 60.51875 69.99 6.726107142857143 1.10825 4.85485 143.19795714285712 October 191.84875 69.99 3.7455785714285716 1.2486964285714286 4.30378

## Well, not exactly helpful

...but you get the point.  This is still a standard Kodexa document with all the basic document properties, but because this represents an Excel document, it does have a few differences that make it different from a base Kodexa document.

The ContentNodes in this freshly parsed document are all one of the follwing types:  worksheet, row, or cell.  Worksheets can have one or more children of type 'row' which may have one or more children of type 'cell'.  Armed with this information, we can do a bit more intelligent inspection on the contents.

Let's start by taking a look at the worksheets in the document:


In [6]:

# This selector will select all of the nodes of type 'worksheet' within the document
all_worksheets = kodexa_doc.select('//worksheet')

print(f'There are {len(all_worksheets)} worksheets in this Excel file')

for w in all_worksheets:
    print(w)

There are 7 worksheets in this Excel file
ContentNode [type:worksheet] (0 features, 32 children) [2019 Reimbursables]
ContentNode [type:worksheet] (0 features, 21 children) [Phone]
ContentNode [type:worksheet] (0 features, 20 children) [ISP]
ContentNode [type:worksheet] (0 features, 20 children) [Power]
ContentNode [type:worksheet] (0 features, 20 children) [Nat Gas]
ContentNode [type:worksheet] (0 features, 20 children) [Water Sewer]
ContentNode [type:worksheet] (0 features, 38 children) [Mileage]


## We have 7 different worksheets in this one Excel file

From the names of the worksheets, it looks like each of them cover a different type of expense.  We're interested in the aggregated expenses, and those are located on the first worksheet, '2019 Reimbursables'.

Let's get the tables from this workbook!

## But wait, aren't all spreadsheets (workbooks) just full of tables?  

Nope, spreadsheets aren't just tables.  As we can see in this example, an Excel workbook (spreadsheet) can contain multiple worksheets each with a multitude of unstructured data.  

A single worksheet can contain multiple table-like structures, interspersed with text, formulas, etc.  There is no schema for how data has to be entered into a spreadsheet, that's why they're considered unstructured (maybe semi-structured) data.

## Let's look a little closer at the first worksheet

We'll iterate over the rows for the first worksheet and see how many child cells each has.


In [7]:

# Selector will get all children of type 'row' from the first worksheet
first_worksheet_rows = all_worksheets[0].select('//row')

for i in range(len(first_worksheet_rows)):
    c = first_worksheet_rows[i]
    print(f'Row {i} :: {len(c.children)}')
    

Row 0 :: 1
Row 1 :: 6
Row 2 :: 7
Row 3 :: 7
Row 4 :: 7
Row 5 :: 7
Row 6 :: 7
Row 7 :: 7
Row 8 :: 7
Row 9 :: 7
Row 10 :: 7
Row 11 :: 7
Row 12 :: 7
Row 13 :: 7
Row 14 :: 7
Row 15 :: 0
Row 16 :: 7
Row 17 :: 0
Row 18 :: 2
Row 19 :: 2
Row 20 :: 2
Row 21 :: 0
Row 22 :: 0
Row 23 :: 0
Row 24 :: 1
Row 25 :: 3
Row 26 :: 3
Row 27 :: 3
Row 28 :: 3
Row 29 :: 3
Row 30 :: 0
Row 31 :: 3


## These rows all have a different number of cells

The fact that many of these rows have differing number of cells indicates that there isn't just one consistent table on this worksheet.

If you haven't taken a look at the document we're using for this example, now may be a good time to do so.  You'll notice that our first sheet in the workbook is named '2019 Reimbursables' and there are (wait for it)...TWO TABLES!  Yup, one for utility expenses and one for mileage reimbursments.  Now it's clear that we can't just take the data from a worksheet and turn it into a table.


## So how do we tag and extract a table?

To locate, tag, and extract tables from our document, we're going to use the 'excel-table-tagger' action.  There are many parameters for this action, but we'll only be using a few of them.  Complete documentation for all the action's parameters can be found here: [Excel Table Tagger](https://developer.kodexa.com/kodexa-platform/actions/kodexa/excel-table-tagger)

* **sheet_name_re:** (required) The name of the sheet on which our data is located
* **header_re:** (required) - a regex value used to locate the header row of the table we're interested in
* **tag_to_apply:** (required) - the tag that will be applied to the elements that are in the detected table

There are two tables on the first worksheet, and we're going to tag both of them.  The header for the first table begins with 'Phone' and the header for the second table begins with 'Quarter'.  We'll need that information to set the parameters for our table taggers.


In [8]:

# We can use our parsed document in the pipeline, so there's no need to reparse the original Excel 
pipeline = Pipeline(kodexa_doc)

# The excel-table-tagger allows us to find the tables we want
# Configuring the header_re regex to specify that 'Phone' is in the header for this table - we'll tag each node in that table with 'utilities'
pipeline.add_step(RemoteAction(slug='kodexa/excel-table-tagger', options={'sheet_name_re': '(2019.*)', 'header_re':'Phone', 'tag_to_apply':'utilities'}))

# Configuring the header_re regex to specify that 'Quarter' is in the header for this table - we'll tag each node in that table with 'mileage'
pipeline.add_step(RemoteAction(slug='kodexa/excel-table-tagger', options={'sheet_name_re': '(2019.*)', 'header_re':'Quarter', 'tag_to_apply':'mileage'}))

# Run it!
pipeline.run()

# Let's get the tagged document
kodexa_doc = pipeline.context.output_document


## Now let's inspect the tagged tables

In [9]:
utility_cells = kodexa_doc.select('//*[tagRegex("utilities")]')

mileage_cells = kodexa_doc.select('//*[tagRegex("mileage")]')

print(f'There are {len(utility_cells)} nodes tagged with the "utilities" tag')
print(f'There are {len(mileage_cells)} nodes tagged with the "mileage" tag')


There are 97 nodes tagged with the "utilities" tag
There are 15 nodes tagged with the "mileage" tag


## Intersting...let's look closer at the utility_cells


In [10]:

# Let's take a look at the first 20 cells and see what those tags look like

for i in range(0,20):
    c = utility_cells[i]
    print(f'{c.type} :: {c.get_tags()}')
    

cell :: ['utilities_col2']
cell :: ['utilities_col3']
cell :: ['utilities_col4']
cell :: ['utilities_col5']
cell :: ['utilities_col6']
cell :: ['utilities_col7']
cell :: ['utilities_col1']
cell :: ['utilities_col2']
cell :: ['utilities_col3']
cell :: ['utilities_col4']
cell :: ['utilities_col5']
cell :: ['utilities_col6']
cell :: ['utilities_col7']
cell :: ['utilities_col1']
cell :: ['utilities_col2']
cell :: ['utilities_col3']
cell :: ['utilities_col4']
cell :: ['utilities_col5']
cell :: ['utilities_col6']
cell :: ['utilities_col7']


## Huh.  So the tag name also includes the column number

Well that's different.  When tagging the cells in a table, the parameter for 'tag_to_apply' is appended with the number of the column 'colN'.  This is useful information as it makes it easy to see how many columns we have in this table...let's look at that now.


In [11]:

# We want to get the unique tag names that have been applied to these 'utility_cells'
# the get_tags() function will return a list of tags.  We only expect one as we've only applied one tag to these cells

u_set = set([c.get_tags()[0] for c in utility_cells])
print(f'\n{u_set}\n')



{'utilities_col6', 'utilities_col5', 'utilities_col3', 'utilities_col7', 'utilities_col1', 'utilities_col2', 'utilities_col4'}



## Cool - so we have 7 columns in our Utilities table!

That's great and all, but what do we do with it?

Working with the tagged data in the cells may be awkard, so we've added the ability to extract this data directly to a Kodexa TableDataStore which is perfect for capturing tabular data such as this.

To create a TableDataStore from our tagged tables, we'll run a pipeline again, this time adding a store



In [12]:


# Again, using the same document.  Since we're re-running the same tagging operation as before, we don't have to worry about excess/errant tags being added
pipeline = Pipeline(kodexa_doc)

# Same options as before, with the addition of the 'store_name' parameter - that's how we'll access that store
pipeline.add_step(RemoteAction(slug='kodexa/excel-table-tagger', 
                               options={'sheet_name_re': '(2019.*)', 'header_re':'Phone', 'tag_to_apply':'utilities', 'store_name' : 'utility_store'}))

# Same options as before, with the addition of the 'store_name' parameter - that's how we'll access that store
pipeline.add_step(RemoteAction(slug='kodexa/excel-table-tagger', 
                               options={'sheet_name_re': '(2019.*)', 'header_re':'Quarter', 'tag_to_apply':'mileage', 'store_name' : 'mileage_store'}))

# Run it!
pipeline.run()

# Let's get the tagged document
kodexa_doc = pipeline.context.output_document


## So what did we get?

We added two store names for the excel-table-tagger actions, so let's see if we can access them

In [14]:

# Print any store names that are avialable on the pipeline context
print(f'\n{pipeline.context.get_store_names()}\n')



dict_keys(['utility_store', 'mileage_store'])



## Cool, cool, cool.  Let's access those stores!

In [15]:

# We'll get each store by name from the pipeline's context
utility_store = pipeline.context.get_store('utility_store')
mileage_store = pipeline.context.get_store('mileage_store')


## Now inspect that data

Each of our stores is a kodexa TableDataStore, and those have a method to convert the store's data to a dataframe.  
Let's call that method and view the data in a nice, pretty format

In [16]:

# Convert the utility_store to a pandas dataframe
utility_store.to_df()


,NaN,Phone,ISP,Electricity,Natural Gas,Water Sewer,Amt due from Business
0,January,60.5,59.99,4.316046428571429,6.192921428571429,2.7588785714285717,133.75784642857144
1,February,60.5,59.99,3.5990035714285713,5.315514285714285,3.0407928571428573,132.4453107142857
2,March,57.79375,59.99,3.7471107142857147,5.297639285714286,2.674610714285714,129.5031107142857
3,April,60.512499999999996,59.99,3.318621428571429,0.6991678571428571,2.495860714285714,127.01615
4,May,60.512499999999996,69.99,3.3875678571428574,1.4126357142857144,2.651117857142857,137.95382142857144
5,June,60.5125,69.99,5.086714285714286,1.0694357142857145,3.8941964285714286,140.55284642857143
6,July,60.7125,69.99,9.156596428571428,0.9213285714285714,5.1970285714285716,145.97745357142855
7,August,60.53125,69.99,7.94365,0.9213285714285714,4.774667857142857,144.16089642857142
8,September,60.51875,69.99,6.726107142857143,1.10825,4.85485,143.19795714285712
9,October,191.84875,69.99,3.7455785714285716,1.2486964285714286,4.303789285714286,271.13681428571425


In [17]:

# Convert the mileage_store to a pandas dataframe
mileage_store.to_df()


,NaN,Quarter,Distance,Amt due from Business
0,None,Q1,1163.9999999999998,675.12
1,None,Q2,716.8000000000001,415.7440000000001
2,None,Q3,79.80000000000001,46.284
3,None,Q4,45.6,26.448
